<a href="https://colab.research.google.com/github/Sanchitraina1999/LP-I/blob/master/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-get update
!apt-get install cuda-9.2

--2021-12-20 11:16:03--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-12-20 11:16:03--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?Tr5mjU-lC1Db323FBTtuVvNYJH2xBU4sdfHv9n4l80y6zw3A1_NeK7cBDaMhokyAASM-6-zHA2qYf_Jz-GsVvFCg_8KGlqwYwm-YD9GAnf7vd

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


A1: **a) Implement Parallel Reduction using Min, Max, Sum and Average
operations.
b) Write a CUDA program that, given an N-element vector, find-
• The maximum element in the vector
• The minimum element in the vector
• The arithmetic mean of the vector
• The standard deviation of the values in the vector
Test for input N and generate a randomized vector V of length N (N should be large). The program should generate output as the two computed maximum values as well as the time taken to find each value.**

In [ ]:
code = """#include "bits/stdc++.h"
using namespace std;

#define N 8
size_t bytes;

__global__ void min(double *array, int n) {
    unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
    int step_size = 1;

    while(n > 0) {
        if(id < n) {
            int i = (int)id * step_size * 2;
            int j = i + step_size;
            if(array[i] > array[j])
                array[i] = array[j];
        }
        step_size = step_size*2;
        n = n/2;
    }
}

__global__ void max(double *array, int n) {
    unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
    int step_size = 1;

    while(n > 0) {
        if(id < n) {
            int i = (int)id * step_size * 2;
            int j = i + step_size;
            if(array[i] < array[j])
                array[i] = array[j];
        }
        step_size = step_size*2;
        n = n/2;
    }
}


__global__ void sum(double *array, int n) {
    unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
    int step_size = 1;

    while(n > 0) {
        if(id < n) {
            int i = (int)id * step_size * 2;
            int j = i + step_size;
            array[i] = array[i] + array[j];
        }
        step_size = step_size*2;
        n = n/2;
    }
}

__global__ void mean_diff_square(double* array, double avg, int n){
    unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
    if(id < n)
        array[id] = (array[id]-avg)*(array[id]-avg);
}

int main() {
    double *array = new double [N];
    cout<<"array: ";
    for(int i=0; i<N; i++){
        array[i] = rand()%97;
        cout<<array[i]<<" ";
    }
    cout<<endl;

    double result;
    double *d_array;
    bytes = N*sizeof(double); //calculate no. of bytes for array
    cudaMalloc(&d_array, bytes);

    //MIN
    cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
    min<<<1,N/2>>>(d_array,N/2);
    cudaDeviceSynchronize();
    cudaMemcpy(&result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
    cout<<"min: "<<result<<endl;

    //MAX
    cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
    max<<<1,N/2>>>(d_array, N/2);
    cudaDeviceSynchronize();
    cudaMemcpy(&result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
    cout<<"max: "<<result<<endl;

    //SUM
    cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
    sum<<<1,N/2>>>(d_array,N/2);
    cudaDeviceSynchronize();
    cudaMemcpy(&result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
    cout<<"sum: "<<result<<endl;

    //AVG
    double avg = result/N;
    cout<<"avg: "<<avg<<endl;

    //STD
    cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
    mean_diff_square<<<1,N>>>(d_array, avg, N);
    cudaDeviceSynchronize();
    sum<<<1,N/2>>>(d_array,N/2);
    cudaDeviceSynchronize();
    cudaMemcpy(&result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
    double variance = result/N;
    cout<<"std: "<<sqrt(variance)<<endl;

    cudaFree(d_array);
    return 0;
}"""

In [ ]:
file = open("main.cu", "w")
file.write(code)
file.close()

In [ ]:
!nvcc main.cu

In [ ]:
!./a.out

array: 46 24 91 22 91 8 80 12 
min: 8
max: 91
sum: 374
avg: 46.75
std: 33.2819


In [ ]:
!nvprof ./a.out

array: 46 24 91 22 91 8 80 12 
==20783== NVPROF is profiling process 20783, command: ./a.out
min: 8
max: 91
sum: 374
avg: 46.75
std: 33.2819
==20783== Profiling application: ./a.out
==20783== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   24.48%  8.7040us         4  2.1760us  2.0160us  2.3360us  [CUDA memcpy DtoH]
                   21.51%  7.6480us         2  3.8240us  3.8080us  3.8400us  sum(double*, int)
                   19.71%  7.0080us         4  1.7520us  1.5040us  2.3360us  [CUDA memcpy HtoD]
                   13.68%  4.8640us         1  4.8640us  4.8640us  4.8640us  max(double*, int)
                   13.59%  4.8320us         1  4.8320us  4.8320us  4.8320us  min(double*, int)
                    7.02%  2.4960us         1  2.4960us  2.4960us  2.4960us  mean_diff_square(double*, double, int)
      API calls:   99.49%  264.16ms         1  264.16ms  264.16ms  264.16ms  cudaMalloc
                    0.19%  

A2:  **Vector and Matrix Operations-
Design parallel algorithm to
Add two large vectors
Multiply Vector and Matrix
Multiply two N × N arrays using n2 processors**


In [ ]:
code="""#include "bits/stdc++.h"
using namespace std;

#define N 8
size_t bytes;

__global__ void vector_addition(int *a, int *b, int *c, int n){
    unsigned int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < n)
        c[idx] = a[idx] + b[idx];
}

int main() {
    int *a = new int[N];
    cout<<"a: ";
    for(int i=0; i<N; i++){
        a[i] = rand()%97;
        cout<<a[i]<<" ";
    }
    cout<<endl;

    int *b = new int[N];
    cout<<"b: ";
    for(int i=0; i<N; i++){
        b[i] = rand()%97;
        cout<<b[i]<<" ";
    }
    cout<<endl;

    bytes = N*sizeof(int);

    int *d_a;
    cudaMalloc(&d_a, bytes);
    cudaMemcpy(d_a, a, bytes, cudaMemcpyHostToDevice);

    int *d_b;
    cudaMalloc(&d_b, bytes);
    cudaMemcpy(d_b, b, bytes, cudaMemcpyHostToDevice);

    int *c = new int[N]{0};
    int *d_c;
    cudaMalloc(&d_c, bytes);

    int threads = 64;
    int blocks = ceil(float(N)/float(threads));

    vector_addition<<<blocks,threads>>>(d_a, d_b, d_c, N);
    cudaDeviceSynchronize();
    cudaMemcpy(c, d_c, bytes, cudaMemcpyDeviceToHost);

    cout<<"c: ";
    for(int i=0; i<N; i++) cout<<c[i]<<" ";
    cout<<endl;

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}
"""

In [ ]:
file = open("main.cu", "w")
file.write(code)
file.close()

In [ ]:
!nvcc main.cu

In [ ]:
!./a.out

a: 46 24 91 22 91 8 80 12 
b: 17 53 50 58 85 17 23 70 
c: 63 77 141 80 176 25 103 82 


In [ ]:
!nvprof ./a.out

a: 46 24 91 22 91 8 80 12 
b: 17 53 50 58 85 17 23 70 
==20868== NVPROF is profiling process 20868, command: ./a.out
c: 63 77 141 80 176 25 103 82 
==20868== Profiling application: ./a.out
==20868== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.97%  3.7760us         2  1.8880us  1.5040us  2.2720us  [CUDA memcpy HtoD]
                   30.21%  2.7840us         1  2.7840us  2.7840us  2.7840us  vector_addition(int*, int*, int*, int)
                   28.82%  2.6560us         1  2.6560us  2.6560us  2.6560us  [CUDA memcpy DtoH]
      API calls:   99.45%  211.42ms         3  70.473ms  2.7950us  211.41ms  cudaMalloc
                    0.26%  559.50us         1  559.50us  559.50us  559.50us  cuDeviceTotalMem
                    0.09%  187.19us        96  1.9490us     122ns  75.994us  cuDeviceGetAttribute
                    0.08%  160.23us         1  160.23us  160.23us  160.23us  cudaLaunchKernel
                   

In [ ]:
code="""#include "bits/stdc++.h"
using namespace std;

#define N 8
#define M 4
size_t bytes;

// 1xN * NxM = 1xM
__global__ void vecmat_multiplication(int* vector, int* matrix, int* result, int n, int m){
    unsigned int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < m){
        int sum = 0;
        for(int i=0; i<n; i++)
            sum += vector[i]*matrix[i*m+idx];

        result[idx] = sum;
    }
}

void print_grid(int* array, int n, int m){
    for(int i=0; i<n; i++){
        for(int j=0; j<m; j++)
            cout<<array[i*m+j]<<" ";
        cout<<endl;
    }
    cout<<endl;
}

int main() {
    int *vector = new int[N];
    for(int i=0; i<N; i++)
        vector[i] = rand()%97;

    int *matrix = new int[N*M];
    for(int i=0; i<N*M; i++)
        matrix[i] = rand()%97;

    cout<<"vector:"<<endl;
    print_grid(vector, 1, N);

    cout<<"matrix:"<<endl;
    print_grid(matrix, N, M);

    int *d_vector;
    bytes = N*sizeof(int);
    cudaMalloc(&d_vector, bytes);
    cudaMemcpy(d_vector, vector, bytes, cudaMemcpyHostToDevice);

    int *d_matrix;
    bytes = N*M*sizeof(int);
    cudaMalloc(&d_matrix, bytes);
    cudaMemcpy(d_matrix, matrix, bytes, cudaMemcpyHostToDevice);

    int *result = new int[M]{0};
    int *d_result;
    bytes = M*sizeof(int);
    cudaMalloc(&d_result, bytes);

    int threads = 64;
    int blocks = ceil(float(M)/float(threads));

    vecmat_multiplication<<<blocks,threads>>>(d_vector, d_matrix, d_result, N, M);
    cudaDeviceSynchronize();

    bytes = M*sizeof(int);
    cudaMemcpy(result, d_result, bytes, cudaMemcpyDeviceToHost);

    cout<<"result:"<<endl;
    print_grid(result, 1, M);

    cudaFree(d_vector);
    cudaFree(d_matrix);
    cudaFree(d_result);
    return 0;
}
"""

In [ ]:
file = open("main.cu", "w")
file.write(code)
file.close()

In [ ]:
!nvcc main.cu

In [ ]:
!./a.out

vector:
46 24 91 22 91 8 80 12 

matrix:
17 53 50 58 
85 17 23 70 
18 49 89 0 
45 81 77 51 
39 60 55 77 
55 8 68 35 
33 93 88 58 
4 72 4 21 

result:
12127 22915 25282 17649 



In [ ]:
!nvprof ./a.out

vector:
46 24 91 22 91 8 80 12 

matrix:
17 53 50 58 
85 17 23 70 
18 49 89 0 
45 81 77 51 
39 60 55 77 
55 8 68 35 
33 93 88 58 
4 72 4 21 

==21011== NVPROF is profiling process 21011, command: ./a.out
result:
12127 22915 25282 17649 

==21011== Profiling application: ./a.out
==21011== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.45%  6.0160us         1  6.0160us  6.0160us  6.0160us  vecmat_multiplication(int*, int*, int*, int, int)
                   31.19%  3.8720us         2  1.9360us  1.6000us  2.2720us  [CUDA memcpy HtoD]
                   20.36%  2.5280us         1  2.5280us  2.5280us  2.5280us  [CUDA memcpy DtoH]
      API calls:   99.38%  197.31ms         3  65.771ms  2.8060us  197.30ms  cudaMalloc
                    0.29%  566.04us         1  566.04us  566.04us  566.04us  cuDeviceTotalMem
                    0.10%  189.91us         1  189.91us  189.91us  189.91us  cudaLaunchKernel
                

In [ ]:
code="""#include "bits/stdc++.h"
using namespace std;

#define M 8
#define N 8
#define K 8
size_t bytes;

// MxN * NxK = MxK
__global__ void matrix_multiplication(int *a, int *b, int *c, int m, int n, int k){
    unsigned int row = blockIdx.y * blockIdx.y + threadIdx.y;
    unsigned int col = blockIdx.x * blockIdx.x + threadIdx.x;
    if(row<m && col<n){
        int sum = 0;
        for(int i=0; i<n; i++)
            sum += a[row*n+i] * b[i*k+col];

        c[row * k + col] = sum;
    }
}

void print_grid(int* array, int n, int m){
    for(int i=0; i<n; i++){
        for(int j=0; j<m; j++)
            cout<<array[i*m+j]<<" ";
        cout<<endl;
    }
    cout<<endl;
}

int main() {
    int *a = new int[M*N];
    for(int i=0; i<M*N; i++)
        a[i] = rand()%97;

    int *b = new int[N*K];
    for(int i=0; i<N*K; i++)
        b[i] = rand()%97;

    cout<<"a:"<<endl;
    print_grid(a,M,N);

    cout<<"b:"<<endl;
    print_grid(b,N,K);

    int *d_a;
    bytes = M*N*sizeof(int);
    cudaMalloc(&d_a, bytes);
    cudaMemcpy(d_a, a, bytes, cudaMemcpyHostToDevice);

    int *d_b;
    bytes = N*K*sizeof(int);
    cudaMalloc(&d_b, bytes);
    cudaMemcpy(d_b, b, bytes, cudaMemcpyHostToDevice);

    int *c = new int[M*K]{0};
    int* d_c;
    bytes = M*K*sizeof(int);
    cudaMalloc(&d_c, bytes);

    unsigned int block_size = 32;
    unsigned int grid_size = ceil(max(M,K)*N/float(block_size));
    dim3 dim_grid(grid_size, grid_size);
    dim3 dim_block(block_size, block_size);

    matrix_multiplication<<<dim_grid,dim_block>>>(d_a, d_b, d_c, M, N ,K);
    cudaDeviceSynchronize();

    bytes = M*K*sizeof(int);
    cudaMemcpy(c, d_c, bytes, cudaMemcpyDeviceToHost);

    cout<<"result:"<<endl;
    print_grid(c,M,K);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}
"""

In [ ]:
file = open("main.cu", "w")
file.write(code)
file.close()

In [ ]:
!nvcc main.cu

In [ ]:
!./a.out

a:
46 24 91 22 91 8 80 12 
17 53 50 58 85 17 23 70 
18 49 89 0 45 81 77 51 
39 60 55 77 55 8 68 35 
33 93 88 58 4 72 4 21 
59 54 13 47 5 68 52 23 
51 44 55 96 60 35 51 2 
29 40 79 18 48 50 54 81 

b:
78 76 73 82 82 77 38 76 
65 51 26 71 53 12 28 8 
88 17 7 51 52 89 84 82 
32 66 3 81 50 88 96 31 
2 73 48 84 84 20 63 53 
5 24 58 59 67 20 67 89 
38 8 74 24 1 61 40 33 
61 75 48 15 66 48 77 2 

result:
17854 16094 16013 22115 19928 21301 22569 20031 
16426 20720 13271 22150 22625 18302 24702 14531 
18953 15050 18215 20666 21351 19560 24801 21295 
19115 19417 14839 23749 20622 22482 24590 16282 
20020 16202 11289 23490 21661 19365 23671 19060 
14489 14699 15079 19167 17272 16220 18080 15518 
17105 19169 14320 26246 21742 22905 25325 19696 
19729 17986 16852 20198 22288 20470 25357 18498 



In [ ]:
!nvprof ./a.out

a:
46 24 91 22 91 8 80 12 
17 53 50 58 85 17 23 70 
18 49 89 0 45 81 77 51 
39 60 55 77 55 8 68 35 
33 93 88 58 4 72 4 21 
59 54 13 47 5 68 52 23 
51 44 55 96 60 35 51 2 
29 40 79 18 48 50 54 81 

b:
78 76 73 82 82 77 38 76 
65 51 26 71 53 12 28 8 
88 17 7 51 52 89 84 82 
32 66 3 81 50 88 96 31 
2 73 48 84 84 20 63 53 
5 24 58 59 67 20 67 89 
38 8 74 24 1 61 40 33 
61 75 48 15 66 48 77 2 

==21076== NVPROF is profiling process 21076, command: ./a.out
result:
17854 16094 16013 22115 19928 21301 22569 20031 
16426 20720 13271 22150 22625 18302 24702 14531 
18953 15050 18215 20666 21351 19560 24801 21295 
19115 19417 14839 23749 20622 22482 24590 16282 
20020 16202 11289 23490 21661 19365 23671 19060 
14489 14699 15079 19167 17272 16220 18080 15518 
17105 19169 14320 26246 21742 22905 25325 19696 
19729 17986 16852 20198 22288 20470 25357 18498 

==21076== Profiling application: ./a.out
==21076== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max

A3. **Parallel Sorting Algorithms-
For Bubble Sort and Merge Sort, based on existing sequential algorithms, design and implement parallel algorithms utilizing all resources available.**

**BUBBLE SORT**

In [ ]:
code="""
#include<omp.h>
#include<iostream>

using namespace std;
void swap(int * num1, int * num2) {
  int temp = * num1;
  * num1 = * num2;
  * num2 = temp;
}
int main() {
  int n = 10;
  int a[n];
  omp_set_num_threads(2);
  for (int i = 0; i < n; i++) {
    a[i] = rand() % 100;
  }
  for (int i = 0; i < n; i++)
    cout << " " << a[i];
  cout << endl;
  int i = 0, j = 0;
  int first = 0;
  double start, end;
  start = omp_get_wtime();
  for (i = 0; i < n - 1; i++) {
    first = i % 2;
    #pragma omp parallel for
    for (j = first; j < n - 1; j++) {
      if (a[j] > a[j + 1])
        swap( & a[j], & a[j + 1]);
    }
  }
  end = omp_get_wtime();
  cout << "Result(parallel) : " << endl;
  for (i = 0; i < n; i++)
    cout << " " << a[i];
  cout << endl;
  cout << "Time parallel = " << (end - start) << endl;
  return 0;
}
"""

In [ ]:
file=open("code.cpp","w")
file.write(code)
file.close()

In [ ]:
!g++ -fopenmp code.cpp

In [ ]:
!./a.out

 83 86 77 15 93 35 86 92 49 21
Result(parallel) : 
 15 21 35 49 77 83 86 86 92 93
Time parallel = 0.000106777


In [ ]:
!nvprof ./a.out

 83 86 77 15 93 35 86 92 49 21
Result(parallel) : 
 15 21 35 49 77 83 86 86 92 93
Time parallel = 0.00623272


**MERGE SORT**

In [ ]:
code="""
#include<iostream>
#include<omp.h>

using namespace std;
void printArray(int * arr, int size) {
  for (int i = 0; i < size; i++) {
    cout << arr[i] << " ";
  }
  cout << endl;
}
void merge(int * arr, int start, int mid, int end) {
  int len = (end - start) + 1;
  int temp[len];
  int cur = 0;
  int i = start;
  int j = mid + 1;
  while (i <= mid && j <= end) {
    if (arr[i] < arr[j]) {
      temp[cur] = arr[i];
      cur++;
      i++;
    } else {
      temp[cur] = arr[j];
      cur++;
      j++;
    }
  }
  if (i <= mid) {
    while (i <= mid) {
      temp[cur] = arr[i];
      i++;
      cur++;
    }
  } else if (j <= end) {
    while (j <= end) {
      temp[cur] = arr[j];
      j++;
      cur++;
    }
  }
  cur = 0;
  for (i = start; i <= end; i++) {
    arr[i] = temp[cur];
    cur++;
  }
}
void mergeSort(int * arr, int start, int end) {
  if (start < end) {
    int mid = (start + end) / 2;
    #pragma omp parallel sections 
    {
      #pragma omp section
      mergeSort(arr, start, mid);
      #pragma omp section
      mergeSort(arr, mid + 1, end);
    }
    merge(arr, start, mid, end);
  }
}
int main(int argc, char * argv[]) {
  int size = 10;
  int a[size];
  double start, end;
  omp_set_num_threads(2);
  for (int i = 0; i < size; i++) {
    a[i] = rand() % 100;
  }
  //int a[]= {7,33,5,5,23,111,75,34,77,121,120};
  for (int i = 0; i < size; i++)
    cout << " " << a[i];
  cout << endl;
  start = omp_get_wtime();
  mergeSort(a, 0, size - 1);
  cout<<"Sorted: ";
  printArray(a, size);
  end = omp_get_wtime();
  cout << "Time parallel = " << (end - start) << endl;
  return 0;
}
"""

In [ ]:
file=open("code.cpp","w")
file.write(code)
file.close()

In [ ]:
!g++ -fopenmp code.cpp

In [ ]:
!./a.out

 83 86 77 15 93 35 86 92 49 21
Sorted: 15 21 35 49 77 83 86 86 92 93 
Time parallel = 0.000159628


In [ ]:
!nvprof ./a.out

 83 86 77 15 93 35 86 92 49 21
Sorted: 15 21 35 49 77 83 86 86 92 93 
Time parallel = 0.000123237


A4. **Parallel Search Algorithm:
Design and implement parallel algorithm utilizing all resources available for
•1. Binary Search for Sorted Array
•2. Best-First Search that (traversal of the graph to reach a target
in the shortest possible path)**

**BINARY SEARCH**

In [11]:
!mpiCC code.cpp
!mpirun --allow-run-as-root -np 4 ./a.out

Position by Process ID 0 = -1 
3 4 
Position by Process ID 1 = -1 
sender 1
13 24 55 56 67 88 
Position by Process ID 3 = 2 
sender 3
7 9 
Position by Process ID 2 = -1 
Found at: 8 by 3sender 2


**BEST_FIRST SEARCH**

In [10]:
!g++ -fopenmp bfs.cpp
!./a.out

Number of Threads = 2
3, 4, 5, 6, 1, 0, 2, 